# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [10]:
# import libraries
import sklearn
import pandas as pd
import numpy as np

import sqlite3

 connect to the database
conn = sqlite3.connect('data/output_etl.db')

 run a query
pd.read_sql('SELECT * FROM df_clean', conn)

In [21]:
# load data from database
# https://auth0.com/blog/sqlalchemy-orm-tutorial-for-python-developers/

from sqlalchemy import create_engine, MetaData, Table
engine = create_engine('sqlite:///data/output_etl.db')

df = pd.read_sql_table("df_clean", engine )
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

In [45]:
# load data from database
# https://auth0.com/blog/sqlalchemy-orm-tutorial-for-python-developers/

X = df["message"]
Y = df.iloc[:,range(4,40)]

In [27]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [47]:
X.head(4)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
Name: message, dtype: object

In [56]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

    
def tokenize(text):
    """
    """
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok, pos="v").strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens



In [57]:
X_tokenized = X.apply( tokenize )

In [58]:
X_tokenized[0]

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

In [134]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, Y_train, Y_test = train_test_split(X_tokenized, Y, random_state=97)

In [88]:
X_train.head()

18717    [walelign, say, 700, quintals, fertilizer, nea...
23463    [mean, fast, track, fund, aid, workers, grind,...
7423                     [note, mark, enough, information]
15191    [government, send, mission, chemical, engineer...
6996              [port, prince, relationship, earthquake]
Name: message, dtype: object

In [135]:
Y_train.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
18717,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23463,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7423,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

fit, transform, predict

Working with text data sklearn tutorial: https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [93]:
vectorizer = CountVectorizer(analyzer=lambda x: x)
X_train2 = vectorizer.fit_transform(X_train).toarray()
#X_train = train[col].apply(lambda x: cv.fit_transform(x))

print(X_train2)
print(X_train2.shape)
print(vectorizer.get_feature_names()[1000:1010])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(19662, 26050)
['6months', '6moswe', '6nsnsnoaso', '6o4lft', '6oeutsvrwalking', '6ogjrx', '6ozdm0fkgood', '6phk58', '6pkpgv', '6pm']


In [185]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer=lambda x: x)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(random_state=97)),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [186]:
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer=lambda x: x)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=97), n_jobs=-1) ),
    #('clf', RandomForestClassifier()),
])

In [138]:
# train classifier
pipeline.fit(X_train, Y_train)

# evaluate all steps on test set
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [164]:
from sklearn.metrics import accuracy_score

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()
    #accuracy2 = accuracy_score(y_test, y_pred)

    print("Labels:", labels, "\n")
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:\n", accuracy, "\n")
    #print("Accuracy:", accuracy2)

In [165]:
display_results(Y_test, Y_pred)

Labels: [0 1 2] 

Accuracy:
 related                   0.810803
request                   0.900977
offer                     0.994965
aid_related               0.778914
medical_help              0.926305
medical_products          0.957278
search_and_rescue         0.973604
security                  0.982911
military                  0.966890
child_alone               1.000000
water                     0.957888
food                      0.938663
shelter                   0.940494
clothing                  0.986878
money                     0.977724
missing_people            0.989167
refugees                  0.967501
death                     0.961855
other_aid                 0.872139
infrastructure_related    0.935307
transport                 0.959719
buildings                 0.952243
electricity               0.983979
tools                     0.995575
hospitals                 0.987794
shops                     0.994660
aid_centers               0.987641
other_infrastructure      

In [171]:
# importing the collections module
import collections
# intializing the arr
# getting the elements frequencies using Counter class
elements_count = collections.Counter(Y_pred[:,1])
# printing the element and the frequency
for key, value in elements_count.items():
   print(f"{key}: {value}")


0: 5856
1: 698


In [100]:
Y_test.value_counts()

1    4925
0    1588
2      41
Name: related, dtype: int64

In [129]:
x_names = df.columns[4:40]
values, keys = list(), list()

for col in x_names:
    values.append( df[col].value_counts().values )
    keys.append( df[col].name )

In [130]:
dict(zip(keys, values))

{'related': array([19906,  6122,   188]),
 'request': array([21742,  4474]),
 'offer': array([26098,   118]),
 'aid_related': array([15356, 10860]),
 'medical_help': array([24132,  2084]),
 'medical_products': array([24903,  1313]),
 'search_and_rescue': array([25492,   724]),
 'security': array([25745,   471]),
 'military': array([25356,   860]),
 'child_alone': array([26216]),
 'water': array([24544,  1672]),
 'food': array([23293,  2923]),
 'shelter': array([23902,  2314]),
 'clothing': array([25811,   405]),
 'money': array([25612,   604]),
 'missing_people': array([25918,   298]),
 'refugees': array([25341,   875]),
 'death': array([25022,  1194]),
 'other_aid': array([22770,  3446]),
 'infrastructure_related': array([24511,  1705]),
 'transport': array([25015,  1201]),
 'buildings': array([24883,  1333]),
 'electricity': array([25684,   532]),
 'tools': array([26057,   159]),
 'hospitals': array([25933,   283]),
 'shops': array([26096,   120]),
 'aid_centers': array([25907,   309

### 6. Improve your model
Use grid search to find better parameters. 

In [189]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [193]:
from sklearn.model_selection import GridSearchCV

# make lists of different parameters to check
parameters = {
    "clf__estimator__max_features": ["sqrt", "log2"],
    "clf__estimator__min_samples_split": [2,3,4,5]
  }

grid_pipeline = GridSearchCV(pipeline, param_grid=parameters)

grid_pipeline.fit(X_train, Y_train)
y_pred_cv = cv.predict(X_test)

KeyboardInterrupt: 

In [138]:
# train classifier
pipeline.fit(X_train, Y_train)

# evaluate all steps on test set
Y_pred = pipeline.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:

pipeline = Pipeline([
    ('features', FeatureUnion([

        ('nlp_pipeline', Pipeline([
            ('vect', CountVectorizer()
            ('tfidf', TfidfTransformer())
        ])),

        ('txt_len', TextLengthExtractor())
    ])),

    ('clf', RandomForestClassifier())
])

# train classifier
pipeline.fit(Xtrain)

# predict on test data
predicted = pipeline.predict(Xtest)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

https://www.geeksforgeeks.org/understanding-python-pickling-example/#:~:text=Python%20pickle%20module%20is%20used,list%2C%20dict%2C%20etc.)

In [180]:
import pickle

# Save to file in the current working directory
filename = 'fitted_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(pipeline, file)

# Load from file
with open(filename, 'rb') as file:
    fitted_model = pickle.load(file)

    #import joblib
#model = joblib.load("../models/your_model_name.pkl")

PicklingError: Can't pickle <function <lambda> at 0x7fe51133e940>: attribute lookup <lambda> on __main__ failed

In [182]:
import joblib
joblib.dump(pipeline, filename)

PicklingError: Can't pickle <function <lambda> at 0x7fe51133e940>: it's not found as __main__.<lambda>

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.